In [44]:
# %pip install sequential
%pip install tensorflow

In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import  classification_report
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from tensorflow.keras import Sequential, Model, Input, models
from tensorflow.keras.layers import Dense, concatenate
from tensorflow.keras.utils import to_categorical, plot_model
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score
df_train = pd.read_csv('https://raw.githubusercontent.com/ThulaniNyama/south-african-language-identification-hack-2022-data/main/train_set.csv')
print(df_train.head())

  lang_id                                               text
0     xho  umgaqo-siseko wenza amalungiselelo kumaziko ax...
1     xho  i-dha iya kuba nobulumko bokubeka umsebenzi na...
2     eng  the province of kwazulu-natal department of tr...
3     nso  o netefatša gore o ba file dilo ka moka tše le...
4     ven  khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [21]:
df_test =pd.read_csv('https://raw.githubusercontent.com/ThulaniNyama/south-african-language-identification-hack-2022-data/main/test_set.csv')
print(df_test.head())

   index                                               text
0      1  Mmasepala, fa maemo a a kgethegileng a letlele...
1      2  Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2      3         Tshivhumbeo tshi fana na ngano dza vhathu.
3      4  Kube inja nelikati betingevakala kutsi titsini...
4      5                      Winste op buitelandse valuta.


In [22]:
#filter languages 
#languages from different alphabet groups (referred to as dif_group). Accuracy: 0.998% 
dataset_1 = df_train[df_train['lang_id'] =='afr']
dataset_2 = df_train[df_train['lang_id'] =='eng']
dataset_3 = df_train[df_train['lang_id'] =='nbl']
dataset_4 = df_train[df_train['lang_id'] =='xho']

dataset_5 = df_train[df_train['lang_id'] =='zul']
dataset_6 = df_train[df_train['lang_id'] =='sot']
dataset_7 = df_train[df_train['lang_id'] =='nso']

dataset_8 = df_train[df_train['lang_id'] =='tsn']
dataset_9 = df_train[df_train['lang_id'] =='ssw']
dataset_10 = df_train[df_train['lang_id'] =='ven']
dataset_11 = df_train[df_train['lang_id'] =='tso']

In [23]:
df_train = pd.concat([dataset_1, dataset_2, dataset_3, dataset_4, dataset_5, dataset_6, dataset_7, dataset_8, dataset_9, dataset_10, dataset_11])

In [24]:
languages = df_train['lang_id'].unique() #get unique language lablels
#sort labels so they always get same indexes 
languages = sorted(list(languages))
print('Unique languages:', languages)

Unique languages: ['afr', 'eng', 'nbl', 'nso', 'sot', 'ssw', 'tsn', 'tso', 'ven', 'xho', 'zul']


In [25]:
X = df_train['text']
y = df_train['lang_id']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.17218, random_state=42) #specific random_state to always get same split results

In [27]:
#count most frequent words in each input sentence
vectorizer1 = CountVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1, 1), max_features=2000)
words_features_train = vectorizer1.fit_transform(X_train).toarray()
words_features_test = vectorizer1.fit_transform(X_test).toarray()

In [28]:
#count most frequent ngrams in each input sentence
vectorizer2 = CountVectorizer(strip_accents='unicode', analyzer='char', ngram_range=(1, 4), max_features=2000)
ngram_features_train = vectorizer2.fit_transform(X_train).toarray()
ngram_features_test = vectorizer2.fit_transform(X_test).toarray()

In [29]:
#Build a dictionary to map unique vocabulary chars and target labels to their indices
output_label_index = dict(zip(languages, range(len(languages))))

In [30]:
#output labels are converted to one-hot vectors
output_labels = [to_categorical(output_label_index[label], num_classes=len(languages)) for label in y_train]
test_output_labels = [to_categorical(output_label_index[label], num_classes=len(languages)) for label in y_test]

In [31]:
#normalize input vectors to be between 0 and 1
train_data_words_unigrams = words_features_train.astype('float32') / words_features_train.max()
train_data_char_unigrams = ngram_features_train.astype('float32') / ngram_features_train.max()
test_data_words_unigrams = words_features_test.astype('float32') / words_features_test.max()
test_data_char_unigrams = ngram_features_test.astype('float32') / ngram_features_test.max()

# print(train_data_words_unigrams[:1])
# print(test_data_char_unigrams[:1])

In [32]:
#the main model layers
words_feature_inputs = Input(shape = (len(words_features_train[0]),), name='words_feature_inputs')
ngram_feature_inputs = Input(shape = (len(ngram_features_train[0]),), name='ngram_feature_inputs')
concatenated_input = concatenate([words_feature_inputs, ngram_feature_inputs])

In [33]:
dense_layer1 = Dense(512, activation='relu')(concatenated_input)
dense_layer2 = Dense(256, activation='relu')(dense_layer1)
dense_layer3 = Dense(128, activation='relu')(dense_layer2)
model_outputs = Dense(len(languages), activation='softmax')(dense_layer3)

In [57]:
#the complete model 
model = Model([words_feature_inputs, ngram_feature_inputs], model_outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])
model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 words_feature_inputs (InputLay  [(None, 2000)]      0           []                               
 er)                                                                                              
                                                                                                  
 ngram_feature_inputs (InputLay  [(None, 2000)]      0           []                               
 er)                                                                                              
                                                                                                  
 concatenate_1 (Concatenate)    (None, 4000)         0           ['words_feature_inputs[0][0]',   
                                                                  'ngram_feature_inputs[0][0

In [58]:
model.fit(
    [train_data_words_unigrams, train_data_char_unigrams], np.array(output_labels),
    epochs=40,
    batch_size=256,
    verbose = 0.9,
    shuffle=True,
    validation_data=([test_data_words_unigrams, test_data_char_unigrams], np.array(test_output_labels))
    # validation_split=0.2
)

Epoch 1/400
Epoch 2/400
Epoch 3/400
Epoch 4/400
Epoch 5/400
Epoch 6/400
Epoch 7/400
Epoch 8/400
Epoch 9/400
Epoch 10/400
Epoch 11/400
Epoch 12/400
Epoch 13/400
Epoch 14/400
Epoch 15/400
Epoch 16/400
Epoch 17/400
Epoch 18/400
Epoch 19/400
Epoch 20/400
Epoch 21/400
Epoch 22/400
Epoch 23/400
Epoch 24/400
Epoch 25/400
Epoch 26/400
Epoch 27/400
Epoch 28/400
Epoch 29/400
Epoch 30/400
Epoch 31/400
Epoch 32/400
Epoch 33/400
Epoch 34/400
Epoch 35/400
Epoch 36/400
Epoch 37/400
Epoch 38/400
Epoch 39/400
Epoch 40/400
Epoch 41/400
Epoch 42/400
Epoch 43/400
Epoch 44/400
Epoch 45/400
Epoch 46/400
Epoch 47/400
Epoch 48/400
Epoch 49/400
Epoch 50/400
Epoch 51/400
Epoch 52/400
Epoch 53/400
Epoch 54/400
Epoch 55/400
Epoch 56/400
Epoch 57/400
Epoch 58/400
Epoch 59/400
Epoch 60/400
Epoch 61/400
Epoch 62/400
Epoch 63/400
Epoch 64/400
Epoch 65/400
Epoch 66/400
Epoch 67/400
Epoch 68/400
Epoch 69/400
Epoch 70/400
Epoch 71/400
Epoch 72/400
Epoch 73/400
Epoch 74/400
Epoch 75/400
Epoch 76/400
Epoch 77/400
Epoch 78

In [36]:
reverse_output_label_index = dict((i, label) for label, i in output_label_index.items())

In [37]:
predictions = model.predict([test_data_words_unigrams, test_data_char_unigrams])
predictions = np.argmax(predictions, axis=-1)
predicted_labels = [reverse_output_label_index[pred] for pred in predictions]

index = 0
for sentence in X_test[:10]:
    print("-" * 50)
    print(sentence)
    print(predicted_labels[index])
    index+=1

accuracy = accuracy_score(y_test, predicted_labels)
print('accuracy:', accuracy)

--------------------------------------------------
izabelo zokukghonakalisa iimfundo kwezokuhlalisana nezomnotho kanye namarhubhululo mayelana nokuthuthukiswa kokusetjenziswa kwamanzi wamaphrojekthi wokulima izabelwezi zifaka kwaphela izinto ezihlobana neemfundo zokusetjenziswa kwamanzi kanye namarhubhululo
nbl
--------------------------------------------------
ke lenyalo la zinhle metswalle ya hae yohle ho tswa seterekeng e tlile moketeng ho na le dipina tse ngata motjeko le ho nwa tse etsahalang sheba sebtshwantsho se katlase ho na le mehlala e ya moo batho ba nwang ka mokgwa o loketseng e bites
sot
--------------------------------------------------
r wa mamiliyoni yi ta vekisiwa eka nhluvukiso wa swipano eka tikhodi hinkwato ta swa mintlangu ngopfu-ngopfu eka tindzhawu leti ti nga ni vusweti swinene ku humesa na ku aka vuswikoti bya hina bya mintlangu na ku antswisa mphikizano eka mintlangu ya hina
nbl
--------------------------------------------------
unakho ukungasamkeli isicelo s

In [40]:
print(predicted_labels[0])

nbl


In [42]:
submission_df = pd.DataFrame(df_test['index'])
submission_df['lang_id'] = predicted_labels
submission_df.to_csv('Final.csv', index=False)

In [55]:
print(classification_report(y_test, predicted_labels))

              precision    recall  f1-score   support

         afr       0.55      0.70      0.61       494
         eng       1.00      0.02      0.03       534
         nbl       0.20      0.94      0.33       510
         nso       0.54      0.69      0.61       497
         sot       0.66      0.87      0.75       516
         ssw       1.00      0.00      0.00       495
         tsn       0.37      0.04      0.07       533
         tso       1.00      0.03      0.05       524
         ven       0.61      0.67      0.64       508
         xho       0.36      0.38      0.37       533
         zul       0.11      0.02      0.04       538

    accuracy                           0.39      5682
   macro avg       0.58      0.40      0.32      5682
weighted avg       0.58      0.39      0.31      5682

